**Quantidade de leitos por município, dados do DataSUS**

Libraries

In [0]:
import pandas as pd
import numpy as np

###Leitura de CSV's

Leitos por município

In [13]:
leitos_internacao = pd.read_csv("https://raw.githubusercontent.com/fraterno20/fraterno20-python-notebooks/master/Dataset/Dados-Primarios/leitos_internacao.csv", sep = ";", encoding = "iso-8859-1")
leitos_internacao.columns = ["city","leitos_internacao_total","leitos_internacao_sus","leitos_internacao_nao_sus"]
leitos_internacao.head(2)

,city,leitos_internacao_total,leitos_internacao_sus,leitos_internacao_nao_sus
0,110001 Alta Floresta D'Oeste,49,43,6
1,110037 Alto Alegre dos Parecis,16,16,-


Dados de Lagitude e Longitude dos municípios 

In [14]:
city_lat_long =  pd.read_csv("https://raw.githubusercontent.com/fraterno20/fraterno20-python-notebooks/master/Dataset/Dados-Primarios/lat_long_municipios.csv")
city_lat_long.drop(columns = ["capital","Unnamed: 0"], inplace = True)
city_lat_long.rename(columns = {"nome":"city"},inplace = True)
city_lat_long.head(2)

,codigo_ibge,city,latitude,longitude,codigo_uf
0,5200050,Abadia de Goiás,-16.7573,-49.4412,52
1,3100104,Abadia dos Dourados,-18.4831,-47.3916,31


Base do IBGE, contendo código do município, nome do estado, do município, e o código do estado

In [15]:
data_ibge = pd.read_csv("https://raw.githubusercontent.com/fraterno20/fraterno20-python-notebooks/master/Dataset/Dados-Primarios/dados_ibge.csv")
data_ibge.drop(columns = ["Mesorregião Geográfica","Nome_Mesorregião","Microrregião Geográfica","Nome_Microrregião","Município","Unnamed: 0"], inplace = True)
data_ibge.rename(columns = {"Nome_Município":"city","Nome_UF":"state","UF":"codigo_uf","Código Município Completo":"codigo_ibge"},inplace = True)
data_ibge.head(2)

,codigo_uf,state,codigo_ibge,city
0,11,Rondônia,1100015,Alta Floresta D'Oeste
1,11,Rondônia,1100379,Alto Alegre dos Parecis


###Manipulação de dataframes

Juntando a tabela da localização geográfica com a dos dados do ibge

In [16]:
city_lat_long = city_lat_long.merge(data_ibge, on = ["city","codigo_ibge","codigo_uf"],sort = "city")
city_lat_long.head(2)

,codigo_ibge,city,latitude,longitude,codigo_uf,state
0,5200050,Abadia de Goiás,-16.7573,-49.4412,52,Goiás
1,3100104,Abadia dos Dourados,-18.4831,-47.3916,31,Minas Gerais


Usando Split para retirar o código do município

In [17]:
new = leitos_internacao["city"].str.split(" ", n = 1, expand = True)
leitos_internacao["codigo_ibge"] = new[0] #Dev sumiu com o ultimo digito de todos os codigos
leitos_internacao.drop(columns=["city"], inplace = True)
leitos_internacao.head(2)

,leitos_internacao_total,leitos_internacao_sus,leitos_internacao_nao_sus,codigo_ibge
0,49,43,6,110001
1,16,16,-,110037


Retirando o último digito para manter a igualdade, já que os dados da datasus vieram faltando o útltimo dígito

In [18]:
city_lat_long["codigo_ibge"] = city_lat_long["codigo_ibge"].map(lambda x: str(x)[:-1])
city_lat_long.head(2)

,codigo_ibge,city,latitude,longitude,codigo_uf,state
0,520005,Abadia de Goiás,-16.7573,-49.4412,52,Goiás
1,310010,Abadia dos Dourados,-18.4831,-47.3916,31,Minas Gerais


Merge com a localização 

In [19]:
leitos_internacao = leitos_internacao.merge(city_lat_long, on = ["codigo_ibge"], sort = "city", how = "outer")
leitos_internacao.head(2)

,leitos_internacao_total,leitos_internacao_sus,leitos_internacao_nao_sus,codigo_ibge,city,latitude,longitude,codigo_uf,state
0,-,-,-,000000,NaN,NaN,NaN,NaN,NaN
1,-,-,-,110000,NaN,NaN,NaN,NaN,NaN


Apagando linha onde tem nulo em state

In [0]:
leitos_internacao.dropna(subset=["state"], inplace = True)

Retirando valores nulos

In [21]:
leitos_internacao.replace("-", 0, inplace = True)
leitos_internacao.replace(np.nan, 0, inplace = True)
leitos_internacao.head(2)

,leitos_internacao_total,leitos_internacao_sus,leitos_internacao_nao_sus,codigo_ibge,city,latitude,longitude,codigo_uf,state
2,49,43,6,110001,Alta Floresta D'Oeste,-11.92830,-61.9953,11.0,Rondônia
3,240,135,105,110002,Ariquemes,-9.90571,-63.0325,11.0,Rondônia


###Download CSV

In [0]:
leitos_internacao.to_csv("leitos_internacao.csv")